# Transfer Learning from MobileNet - Dog vs Cats vs Horses
This notebook contains the code that support [the blog on transfer learning.](https://towardsdatascience.com/keras-transfer-learning-for-beginners-6c9b8b7143e) 

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing the libraries

In [9]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from tensorflow.keras.optimizers import Adam

## Imports The MobileNet Model
Imports the mobilenet model and **discards the last 1000 neuron layer**.

In [3]:
base_model=MobileNet(weights='imagenet',include_top=False) 

17235968/17225924 [==============================] - 0s 0us/step


# Add layers of our own
We add dense layers so that the model can learn more complex functions and classify for better results. 

In [4]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(3,activation='softmax')(x) # 3 -> 4

# base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

# x=base_model.output
# x=GlobalAveragePooling2D()(x)
# x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
# x=Dense(1024,activation='relu')(x) #dense layer 2
# x=Dense(512,activation='relu')(x) #dense layer 3
# preds=Dense(3,activation='softmax')(x) #final layer with softmax activation

*   Specify the inputs
*   Specify the outputs

Now a model has been created based on our architecture 🏆

In [5]:
model=Model(inputs=base_model.input,outputs=preds)
# model=Model(inputs=base_model.input,outputs=preds)
# #specify the inputs
# #specify the outputs
# #now a model has been created based on our architecture

Layers & models also feature a boolean attribute `trainable`. Its value can be changed.
Setting `layer.trainable` to `False` moves all the layer's weights from trainable to
non-trainable.  This is called "freezing" the layer: the state of a frozen layer won't
be updated during training (either when training with `fit()` or when training with
 any custom loop that relies on `trainable_weights` to apply gradient updates).<br> 
 When a trainable weight becomes non-trainable, its value is no longer updated during
 training.

**Example: setting `trainable` to `False`**<br><br>
**Very Importent** : The setting of `trainable` are **Recursive** <br>
If you set `trainable = False` on a model or on any layer that has sublayers,
all children layers become non-trainable as well.


In [6]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

## Data Preprocessing of Our Data


In [10]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/transfer-learning/train', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 197 images belonging to 3 classes.


### Compiling the NN

In [11]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


### Training

In [12]:
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=5)

# model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# # Adam optimizer
# # loss function will be categorical cross entropy
# # evaluation metric will be accuracy

# step_size_train=train_generator.n//train_generator.batch_size
# model.fit_generator(generator=train_generator,
#                    steps_per_epoch=step_size_train,
#                    epochs=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
6/6 [==============================] - 33s 4s/step - loss: 0.9207 - accuracy: 0.7135
Epoch 2/5
6/6 [==============================] - 22s 4s/step - loss: 0.1835 - accuracy: 0.9273
Epoch 3/5
6/6 [==============================] - 22s 3s/step - loss: 0.0282 - accuracy: 0.9818
Epoch 4/5
6/6 [==============================] - 22s 3s/step - loss: 0.0864 - accuracy: 0.9697
Epoch 5/5
6/6 [==============================] - 25s 4s/step - loss: 0.3350 - accuracy: 0.9740


In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv1 (Conv2D)              (None, None, None, 32)    864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, None, None, 32)   128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, None, None, 32)    0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)   288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, None, None, 32)   128       
 ation)                                                      

## Single Prediction

In [14]:
in_classes = {y:x for x,y in train_generator.class_indices.items()}
in_classes

{0: 'cats', 1: 'dogs', 2: 'horses'}

In [31]:
from keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/transfer-learning/single/dog2.jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
result = np.argmax(result)
print(in_classes[result])


dogs
